In [4]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import collections

### Step 1: Load Pretrained Model

In [5]:
# Load Pretrained Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model initially

# Add task-specific layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # Replace 4 with the number of your classes
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Step 2: Prepare Your Dataset

In [6]:
# Set the base directory
base_dir = os.getcwd()  # Get the current working directory

# Construct the path to the dataset
dataset_dir = os.path.join(base_dir, 'dataset2')

# Construct the path to the training directory
train_dir = os.path.join(dataset_dir, 'train')

# Construct the path to the validation directory
val_dir = os.path.join(dataset_dir, 'val')

# Data augmentation for training
data_augmentation = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# No augmentation for validation, just rescaling
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Prepare generators
train_generator = data_augmentation.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 1955 images belonging to 2 classes.
Found 197 images belonging to 2 classes.


### Step 3: Train the Model

In [7]:
# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the new layers initially using train_generator and val_generator
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[early_stopping],
    verbose=1
)

# Fine-tune the entire model
base_model.trainable = True  # Unfreeze the base layers

# Recompile with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the entire model
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[early_stopping],
    verbose=1
)

# Save the trained model
model.save('two_categories_model.keras')


Epoch 1/15


2025-02-02 19:48:12.836076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


62/62 [==============================] - ETA: 0s - loss: 0.3737 - accuracy: 0.8465

2025-02-02 19:49:08.382108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


62/62 [==============================] - 61s 948ms/step - loss: 0.3737 - accuracy: 0.8465 - val_loss: 0.2789 - val_accuracy: 0.8985
Epoch 2/15
62/62 [==============================] - 62s 999ms/step - loss: 0.2814 - accuracy: 0.8926 - val_loss: 0.2659 - val_accuracy: 0.9086
Epoch 3/15
62/62 [==============================] - 69s 1s/step - loss: 0.2358 - accuracy: 0.9090 - val_loss: 0.2745 - val_accuracy: 0.8934
Epoch 4/15
62/62 [==============================] - 71s 1s/step - loss: 0.2147 - accuracy: 0.9212 - val_loss: 0.2827 - val_accuracy: 0.8985
Epoch 5/15
62/62 [==============================] - 71s 1s/step - loss: 0.1963 - accuracy: 0.9212 - val_loss: 0.2696 - val_accuracy: 0.9086
Epoch 6/15
62/62 [==============================] - 76s 1s/step - loss: 0.2007 - accuracy: 0.9233 - val_loss: 0.2626 - val_accuracy: 0.9086
Epoch 7/15
62/62 [==============================] - 73s 1s/step - loss: 0.2060 - accuracy: 0.9217 - val_loss: 0.2548 - val_accuracy: 0.9137
Epoch 8/15
62/62 [=======

2025-02-02 20:03:47.607547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


62/62 [==============================] - ETA: 0s - loss: 0.2774 - accuracy: 0.8905

2025-02-02 20:08:55.113501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


62/62 [==============================] - 314s 5s/step - loss: 0.2774 - accuracy: 0.8905 - val_loss: 0.5874 - val_accuracy: 0.8832
Epoch 2/15
62/62 [==============================] - 301s 5s/step - loss: 0.1519 - accuracy: 0.9458 - val_loss: 0.6596 - val_accuracy: 0.8782
Epoch 3/15
62/62 [==============================] - 303s 5s/step - loss: 0.1087 - accuracy: 0.9586 - val_loss: 1.7519 - val_accuracy: 0.7411
Epoch 4/15
62/62 [==============================] - 236s 4s/step - loss: 0.0929 - accuracy: 0.9678 - val_loss: 0.9967 - val_accuracy: 0.8528
Epoch 5/15
62/62 [==============================] - 170s 3s/step - loss: 0.0784 - accuracy: 0.9708 - val_loss: 0.5123 - val_accuracy: 0.9289
Epoch 6/15
62/62 [==============================] - 234s 4s/step - loss: 0.0586 - accuracy: 0.9790 - val_loss: 0.7411 - val_accuracy: 0.9086
Epoch 7/15
62/62 [==============================] - 193s 3s/step - loss: 0.0455 - accuracy: 0.9836 - val_loss: 0.6560 - val_accuracy: 0.9239
Epoch 8/15
62/62 [======

### Step 4: Real-Time Webcam Prediction

In [8]:
# Load the saved model
model = tf.keras.models.load_model('two_categories_model.keras')

# Class labels for predictions
class_labels = ['Headtop', 'No headwear']

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Define the target size for the images
target_size = (224, 224)

# Initialize a buffer for smoothing predictions
predictions_buffer = collections.deque(maxlen=10)

print("Press 'q' to exit the video feed.")

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video frame. Exiting...")
            break

        # Preprocess the frame
        resized_frame = cv2.resize(frame, target_size)  # Resize to match model input
        img_array = np.expand_dims(resized_frame, axis=0) / 255.0  # Normalize and add batch dimension

        # Make prediction
        predictions = model.predict(img_array, verbose=0)
        class_index = np.argmax(predictions[0])
        prediction_label = class_labels[class_index]
        confidence = predictions[0][class_index] * 100

        # Add prediction to the buffer
        predictions_buffer.append(class_index)

        # Smooth predictions using majority voting
        smoothed_prediction = max(set(predictions_buffer), key=predictions_buffer.count)
        smoothed_label = class_labels[smoothed_prediction]

        # Display the prediction on the frame
        cv2.putText(frame, f"{smoothed_label} ({confidence:.2f}%)", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (75, 75, 75), 2)

        # Show the frame
        cv2.imshow('Hat Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("\nProgram interrupted by the user. Exiting...")

finally:
    # Release the webcam and close windows
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released, video window closed.")

Press 'q' to exit the video feed.
Failed to capture video frame. Exiting...
Resources released, video window closed.


[ WARN:0@4986.657] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index
